In [1]:
import django
import os

os.environ["DJANGO_SETTINGS_MODULE"] = 'imdb.settings'
django.setup()

In [21]:
from DatabaseQueries.models import *
from django.db.models import Subquery, OuterRef, Sum, FloatField, Max, Count, IntegerField, Min, Avg
from django.db.models.functions import Cast
from django.db.models.functions import ExtractMonth

In [3]:
def get_top_10_movies_with_avg_rating():
    top_10_movies = MovieRating.objects.values('movie_id', 'movie__title').annotate(avg_rating=Cast(Sum(F('no_of_ratings') * F('rating_no') * 1.0) / Sum(F('no_of_ratings')), FloatField()))
    return top_10_movies.values('movie__title','avg_rating').order_by('-avg_rating')[:10]
get_top_10_movies_with_avg_rating()

<QuerySet [{'movie__title': 'E', 'avg_rating': 3.3684210526315788}, {'movie__title': 'C', 'avg_rating': 3.1333333333333333}, {'movie__title': 'A', 'avg_rating': 3.0652173913043477}, {'movie__title': 'J', 'avg_rating': 2.9791666666666665}, {'movie__title': 'F', 'avg_rating': 2.9782608695652173}, {'movie__title': 'H', 'avg_rating': 2.9411764705882355}, {'movie__title': 'G', 'avg_rating': 2.9361702127659575}, {'movie__title': 'K', 'avg_rating': 2.9166666666666665}, {'movie__title': 'D', 'avg_rating': 2.857142857142857}, {'movie__title': 'I', 'avg_rating': 2.7916666666666665}]>

In [4]:
def get_top_5_and_least_5_actors():
    top_5_actors = Actor.objects.all().annotate(no_of_movies_acted=Count('moviecast')).order_by('-no_of_movies_acted').values('id')[:5]
    least_5_actors = Actor.objects.all().annotate(no_of_movies_acted=Count('moviecast')).order_by('no_of_movies_acted').values('id')[:5]
    no_of_movies_acted = Actor.objects.filter(Q(id__in=Subquery(top_5_actors)) | Q(id__in=Subquery(least_5_actors))).values_list('name')
    return no_of_movies_acted
get_top_5_and_least_5_actors()

<QuerySet [('Pavan Kalyan',), ('Prabhas',), ('Ram Charan',), ('Rashi',), ('Deepika',), ('Nithin',), ('Bala Krishna',), ('Samantha',), ('Tamanna',)]>

In [5]:
def star_movies():
    star_month = MovieCast.objects.filter(movie_id=OuterRef('movie_id')).values('cast__birth_date__month').annotate(month_count=Count("id")).values_list('cast__birth_date__month', flat=True).order_by('-month_count', 'cast__birth_date__month')
    return MovieCast.objects.values('movie_id','cast__birth_date__month').annotate(month_count=Count("id"), star_month=Subquery(star_month[:1])).filter(movie__release_date__month=F('star_month'), cast__birth_date__month=F('star_month')).order_by('-month_count').values("movie_id","month_count")
star_movies()

<QuerySet [{'movie_id': 1, 'month_count': 2}, {'movie_id': 4, 'month_count': 2}, {'movie_id': 7, 'month_count': 1}]>

In [6]:
def get_actors_movie_count_released_in_their_birth_month():
    return MovieCast.objects.annotate(release_month=ExtractMonth('movie__release_date'), birth_month=ExtractMonth('cast__birth_date')).filter(release_month=F('birth_month')).values("cast__name").annotate(movie_count=Count("movie"))
get_actors_movie_count_released_in_their_birth_month()    

<QuerySet [{'cast__name': 'Deepika', 'movie_count': 1}, {'cast__name': 'NTR', 'movie_count': 1}, {'cast__name': 'Pavan Kalyan', 'movie_count': 1}, {'cast__name': 'Samantha', 'movie_count': 1}, {'cast__name': 'Tamanna', 'movie_count': 1}]>

In [7]:
def difference_between_1star_and_5star():
    movie_difference = MovieRating.objects.filter(movie_id=OuterRef('movie_id')).values('movie_id').annotate(one_star=Sum('no_of_ratings', filter=Q(rating_no=1)), five_star=Sum('no_of_ratings', filter=Q(rating_no=5)), difference=F('one_star') - F('five_star')).values("difference")
    return MovieCast.objects.values("movie_id").annotate(difference=Subquery(movie_difference)).values("cast_id").annotate(difference_sum=Sum('difference')).order_by("-difference_sum")
difference_between_1star_and_5star()

<QuerySet [{'cast_id': 4, 'difference_sum': 14}, {'cast_id': 12, 'difference_sum': 9}, {'cast_id': 5, 'difference_sum': 8}, {'cast_id': 10, 'difference_sum': 7}, {'cast_id': 2, 'difference_sum': 6}, {'cast_id': 11, 'difference_sum': 4}, {'cast_id': 13, 'difference_sum': 2}, {'cast_id': 8, 'difference_sum': -1}, {'cast_id': 9, 'difference_sum': -1}, {'cast_id': 1, 'difference_sum': -2}, {'cast_id': 3, 'difference_sum': -5}]>

In [8]:
from django.db.models import F, ExpressionWrapper, fields
def youngest_cast_age():
    age = ExpressionWrapper(F('movie__release_date') - F('cast__birth_date'), output_field=fields.DurationField())
    youngest_age = MovieCast.objects.filter(movie_id=OuterRef('id')).values('movie_id').annotate(age=age).order_by("age").values("age")[:1]
    return Movie.objects.annotate(youngest_age=Subquery(youngest_age)).values("id","youngest_age").order_by('youngest_age')[:10]
youngest_cast_age()

<QuerySet [{'id': 9, 'youngest_age': 389145600000000}, {'id': 7, 'youngest_age': 407030400000000}, {'id': 10, 'youngest_age': 416016000000000}, {'id': 8, 'youngest_age': 419731200000000}, {'id': 3, 'youngest_age': 428803200000000}, {'id': 1, 'youngest_age': 440294400000000}, {'id': 2, 'youngest_age': 451267200000000}, {'id': 11, 'youngest_age': 454291200000000}, {'id': 5, 'youngest_age': 456883200000000}, {'id': 4, 'youngest_age': 472089600000000}]>

In [9]:
def year_in_which_most_cast_movies_released():
    return MovieCast.objects.values('movie__release_date__year').annotate(year_count=Count('movie__release_date__year')).order_by('-year_count')[:1]
year_in_which_most_cast_movies_released()

<QuerySet [{'movie__release_date__year': 2012, 'year_count': 11}]>

In [10]:
def best_twin_stars():
    return MovieCast.objects.filter(movie__moviecast__movie__id=F('movie_id')).annotate(cast_to_id=F('movie__moviecast__cast__id')).filter(cast_id__lt=F('cast_to_id')).values("cast_id","cast_to_id").annotate(twin_count=Count("id")).order_by("-twin_count")[:1]
best_twin_stars()

<QuerySet [{'cast_id': 4, 'cast_to_id': 12, 'twin_count': 2}]>

In [32]:
def youngest_and_oldest_movies_by_average_age_of_cast():
    age = ExpressionWrapper(F('movie__release_date') - F('cast__birth_date'), output_field=fields.DurationField())
    movie_avg_age = MovieCast.objects.values('movie_id').annotate(average_age=Avg(age))
    youngest_movies = movie_avg_age.order_by('average_age').values('movie_id')[:5]
    oldest_movies = movie_avg_age.order_by('-average_age').values('movie_id')[:5]
    return MovieCast.objects.filter(Q(movie_id__in=(Subquery(youngest_movies))) | Q(movie_id__in=(Subquery(oldest_movies)))).values("movie__title").distinct()
youngest_and_oldest_movies_by_average_age_of_cast()

<QuerySet [{'movie__title': 'B'}, {'movie__title': 'C'}, {'movie__title': 'H'}, {'movie__title': 'J'}, {'movie__title': 'K'}, {'movie__title': 'A'}, {'movie__title': 'E'}, {'movie__title': 'F'}, {'movie__title': 'G'}, {'movie__title': 'I'}]>